# Chipotle Food Recommendation using Apriori Algorithm

### Read Data

In [1]:
import pandas as pd 
data_url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"

In [253]:
df = pd.read_csv(data_url, sep='\t')
df.head(3)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39


## Prepare data for appriori algorithm

Apriori needs products in the chart to be grouped in list.<br>
Ex. [['Chips and Fresh Tomato Salsa', 'Izze', 'Nantucket Nectar', 'Chips and Tomatillo-Green Chili Salsa'],<br>            ['Izze', 'Nantucket Nectar']]

In [58]:
def prepare_data(df):
    current_order_id = df.iloc[0,1]
    carts = []
    products = []
    for row in df.iterrows():
        order_id = row[1]['order_id']
        product_name = row[1]['item_name']
        if  order_id == current_order_id:
            products.append(product_name)
        else:
            carts.append(products)
            products = []
            products.append(product_name)
            current_order_id = order_id
    carts.append(products)
    
    return pd.DataFrame(carts)

In [59]:
carts_df = prepare_data(df)

In [251]:
transactions = []
for i in range(carts_df.shape[0]):
    transactions.append([str(carts_df.values[i,j]) for j in range(12)])
print(f'----Sameple cart ---- \n{transactions[0]}')

----Sameple cart ---- 
['Chips and Fresh Tomato Salsa', 'Izze', 'Nantucket Nectar', 'Chips and Tomatillo-Green Chili Salsa', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']


## Major components of Apriori Algorithm
### Support
support refers to the popularity of item and can be calculated by finding the number of transactions containing a particular item divided by the total number of transactions.

### Confidence
Confidence refers to the likelihood that an item B is also bought if item A is bought. It can be calculated by finding the number of transactions where A and B are bought together, divided by the total number of transactions where A is bought. Mathematically, it can be represented as:

Confidence(A → B) = (Transactions containing both (A and B))/(Transactions containing A)

### Association rule by Lift

lift = 1 → There is no association between A and B.

lift < 1→ A and B are unlikely to be bought together.

lift > 1 → greater the lift greater is the likelihood of buying both products together.

In [263]:
from apyori import apriori
rules = apriori(transactions= transactions, 
                min_support = 0.003, #Support shows transactions with items purchased together in a single transaction.
                min_confidence = 0.3, #Confidence shows transactions where the items are purchased one after the other.
                min_lift=3,
                min_length=2,
                max_length=7
               )

In [264]:
results = list(rules)
print(f'Number of Rules = {len(results)}')

Number of Rules = 12


In [265]:
print('----------- Sample rule --------------')
results[0]

----------- Sample rule --------------


RelationRecord(items=frozenset({'Barbacoa Burrito', 'Chicken Bowl', 'Chips'}), support=0.004907306434023991, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Barbacoa Burrito', 'Chicken Bowl'}), items_add=frozenset({'Chips'}), confidence=0.45, lift=3.9677884615384618)])

## Visualize the predictions

In [266]:
def inspect(results):
    lhs         = [" + ".join(tuple(result[2][0][0])) for result in results]
    rhs         = [" + ".join(tuple(result[2][0][1])) for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

In [267]:

resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['In Cart', 'Recommendation', 
                                                                'Support', 'Confidence', 'Lift'])
resultsinDataFrame.sort_values(by='Lift', ascending=False)

,In Cart,Recommendation,Support,Confidence,Lift
3,Canned Soda + Chicken Bowl,Side of Chips,0.003817,0.411765,7.476995
9,Canned Soda + Chicken Bowl,Side of Chips + None,0.003817,0.411765,7.476995
5,Chips and Tomatillo Red Chili Salsa,Chicken Bowl + Canned Soft Drink,0.008179,0.326087,5.387779
11,Chips and Tomatillo Red Chili Salsa,Chicken Bowl + None + Canned Soft Drink,0.008179,0.326087,5.387779
1,Canned Soda + Chicken Bowl,Bottled Water,0.003272,0.352941,4.203209
7,Canned Soda + Chicken Bowl,Bottled Water + None,0.003272,0.352941,4.203209
6,Barbacoa Burrito + Chicken Bowl,None + Chips,0.004907,0.450000,3.986957
0,Barbacoa Burrito + Chicken Bowl,Chips,0.004907,0.450000,3.967788
8,Chicken Bowl + Bottled Water,None + Chips,0.015267,0.405797,3.595323
2,Chicken Bowl + Bottled Water,Chips,0.015267,0.405797,3.578038
